# Build et création des dossiers/fichiers nécessaire

In [1]:
import os
import subprocess

import experimentsDef as expDef

## défénition des différents executables et expériences

In [2]:
implementation = expDef.getImplementationDetails()
print(implementation)
experimentsHandDetails = expDef.getExpDetails()
print(experimentsHandDetails)

# automatic combinaison of experiments
array_g = [500, 5000, 50000] # size of DNA
array_w_and_h = [32, 64, 128, 256, 1024] # size of the world
array_mutation_rate = [0.0001, 0.00001, 0.000001] # mutation rate
array_exec_type = ["nameCuda", "nameCpu"] # execution

{'baseline': {'folder': 'micro-aevol2', 'nameCuda': 'micro_aevol_gpu', 'nameCpu': 'micro_aevol_cpu'}, 'openMPMutationV1': {'folder': 'openMPMutationV1', 'nameCuda': 'micro_aevol_gpu', 'nameCpu': 'micro_aevol_cpu'}, 'openMPMutationV2': {'folder': 'openMPMutationV2', 'nameCuda': 'micro_aevol_gpu', 'nameCpu': 'micro_aevol_cpu'}}
{'baseline': {'perDefaultCuda': {'executable': 'nameCuda', 'arg': []}, 'CudaWandH64': {'executable': 'nameCuda', 'arg': ['-w', '64', '-h', '64']}, 'perDefaultCPU': {'executable': 'nameCpu', 'arg': []}, 'chargeCPU': {'executable': 'nameCpu', 'arg': ['-w', '128', '-h', '128', '-g', '5000']}, 'chargeMoreRateCPU': {'executable': 'nameCpu', 'arg': ['-w', '128', '-h', '128', '-g', '5000', '-m', '0.001']}, 'chargeLessRateCPU': {'executable': 'nameCpu', 'arg': ['-w', '128', '-h', '128', '-g', '5000', '-m', '0.000001']}}, 'openMPMutationV1': {'perDefaultCPU': {'executable': 'nameCpu', 'arg': []}, 'chargeCPU': {'executable': 'nameCpu', 'arg': ['-w', '128', '-h', '128', '-g'

## build des executables

In [3]:
for exec in implementation.keys():
    # create the folder for the build
    pathBuild = os.path.join("aevol", implementation[exec]["folder"], "build")
    if not os.path.exists(pathBuild):
        os.makedirs(pathBuild)
    else:
        # clean the folder
        subprocess.run(["rm", "-rf", pathBuild])
        os.makedirs(pathBuild)
    # build the executable
    subprocess.run(["cmake", "..", "-DUSE_CUDA=on", "-DDO_TRACES=on", "-DUSE_OMP=on"], cwd=pathBuild)
    subprocess.run(["make"], cwd=pathBuild)


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Traces are activated
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found CUDA: /us

## execution des expériences

In [3]:
def execExp(nameExec, exp):
    workDir = os.getcwd()
    print("Experiment : " + exec + " - " + exp)
    pathBuild = os.path.join(workDir, "aevol", implementation[exec]["folder"], "build")
    pathResults = os.path.join(workDir, "simulations", exec, exp)
    nameExec = implementation[exec][experimentsHandDetails[exec][exp]["executable"]]

    if len(os.listdir(pathResults)) == 0:
        subprocess.run([os.path.join(pathBuild, nameExec), *experimentsHandDetails[exec][exp]["arg"]], cwd=pathResults, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Result of handmade experiments

In [5]:
workDir = os.getcwd()
# creation of the folder for the results
for exec in implementation.keys():
    # create the folder for the results
    pathExperiment = os.path.join(workDir, "simulations", exec)
    if not os.path.exists(pathExperiment):
        os.makedirs(pathExperiment)

    for folder in experimentsHandDetails[exec].keys():
        pathResults = os.path.join(pathExperiment, folder)
        if not os.path.exists(pathResults):
            os.makedirs(pathResults)

# execution of the experiments
for exec in implementation:
    for exp in experimentsHandDetails[exec].keys():
        execExp(exec, exp)

Experiment : baseline - perDefaultCuda
Experiment : baseline - CudaWandH64
Experiment : baseline - perDefaultCPU
Experiment : baseline - chargeCPU
Experiment : baseline - chargeMoreRateCPU
Experiment : baseline - chargeLessRateCPU
Experiment : openMPMutationV1 - perDefaultCPU
Experiment : openMPMutationV1 - chargeCPU
Experiment : openMPMutationV1 - chargeMoreRateCPU
Experiment : openMPMutationV1 - chargeLessRateCPU
Experiment : openMPMutationV2 - perDefaultCPU
Experiment : openMPMutationV2 - chargeCPU
Experiment : openMPMutationV2 - chargeMoreRateCPU
Experiment : openMPMutationV2 - chargeLessRateCPU


# Result of automatic experiments

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [6]:
# generation of the combination
totalExp = len(array_g) * len(array_w_and_h) * len(array_mutation_rate) * len(array_exec_type)
columnsCSV = ["Gen","Resource","Stamp","TimeStamp_Start","TimeStamp_End","Duration","Indiv_id", "exec", "w", "h", "g", "mutation_rate"]
workDir = os.getcwd()
for exec in implementation.keys():
    # create auto experiments folder
    pathAuto = os.path.join(workDir, "simulations", exec, "auto")
    if not os.path.exists(pathAuto):
        os.makedirs(pathAuto)

    i = 1
    # generation of the combinaison
    for exec_type in array_exec_type:
        for g in array_g:
            for w in array_w_and_h:
                for mutation_rate in array_mutation_rate:
                    exp = "AutoType" + str(exec_type) + "W" + str(w) + "G" + str(g) + "MutationRate" + str(mutation_rate)

                    pathBuild = os.path.join(workDir, "aevol", implementation[exec]["folder"], "build")
                    pathResults = os.path.join(workDir, "simulations", exec, "auto", exp)

                    # create the folder for the results
                    if not os.path.exists(pathResults):
                        os.makedirs(pathResults)

                    # execution of the experiment
                    if len(os.listdir(pathResults)) == 0:
                        nameExec = implementation[exec][exec_type]
                        print("Experiment auto : " + exec + " - " + exp + "(" + str(i) + "/" + str(totalExp) + ")")
                        subprocess.run([os.path.join(pathBuild, nameExec), "-g", str(g), "-w", str(w), "-h", str(w), "-m", str(mutation_rate)], cwd=pathResults, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
                        
                    i += 1
    # aggregate the results
    aggregate = pd.DataFrame(data=None, index=None, columns=columnsCSV)
    for exec_type in array_exec_type:
        for g in array_g:
            for w_and_h in array_w_and_h:
                for mutation_rate in array_mutation_rate:
                    # get the name of the executable
                    expName = "AutoType" + str(exec_type) + "W" + str(w_and_h) + "G" + str(g) + "MutationRate" + str(mutation_rate)
                    pathResults = os.path.join(workDir, "simulations", exec, "auto", expName)
                    # add the results to the aggregate (if it exists, their is probleme for some extrem value, otherwise skip)
                    if os.path.exists(os.path.join(pathResults, "trace.csv")):
                        # get the results
                        results = pd.read_csv(os.path.join(pathResults, "trace.csv"), sep=",")
                        # if there is no ligne, add empty one
                        if len(results.index) == 0:
                            results = pd.DataFrame(data=None, index=None, columns=columnsCSV)
                            results = results.append(pd.Series(), ignore_index=True)
                        results["exec"] = exec_type
                        results["w"] = w_and_h
                        results["h"] = w_and_h
                        results["g"] = g
                        results["mutation_rate"] = mutation_rate
                        aggregate = aggregate.append(results, ignore_index=True)
                    else:
                        # add ligne with none for data
                        tmp = pd.DataFrame(data=None, index=None, columns=columnsCSV)
                        tmp = tmp.append(pd.Series(), ignore_index=True)
                        tmp["exec"] = exec_type
                        tmp["w"] = w_and_h
                        tmp["h"] = w_and_h
                        tmp["g"] = g
                        tmp["mutation_rate"] = mutation_rate
                        aggregate = aggregate.append(tmp, ignore_index=True)
    aggregate.to_csv(os.path.join(workDir, "simulations", exec, "auto", "aggregate.csv"), sep=",", index=False)

    # delete all the experiments folder
    for exec_type in array_exec_type:
        for g in array_g:
            for w_and_h in array_w_and_h:
                for mutation_rate in array_mutation_rate:
                    # get the name of the executable
                    expName = "AutoType" + str(exec_type) + "W" + str(w_and_h) + "G" + str(g) + "MutationRate" + str(mutation_rate)
                    pathResults = os.path.join(workDir, "simulations", exec, "auto", expName)
                    # delete the folder
                    if os.path.exists(pathResults):
                        os.rmdir(pathResults)

                    
                    

Experiment auto : baseline - AutoTypenameCudaW1024G5000MutationRate0.0001(28/90)
Experiment auto : baseline - AutoTypenameCudaW1024G5000MutationRate1e-05(29/90)
Experiment auto : baseline - AutoTypenameCudaW1024G5000MutationRate1e-06(30/90)
Experiment auto : baseline - AutoTypenameCudaW256G50000MutationRate0.0001(40/90)
Experiment auto : baseline - AutoTypenameCudaW256G50000MutationRate1e-05(41/90)
Experiment auto : baseline - AutoTypenameCudaW256G50000MutationRate1e-06(42/90)
Experiment auto : baseline - AutoTypenameCudaW1024G50000MutationRate0.0001(43/90)
Experiment auto : baseline - AutoTypenameCudaW1024G50000MutationRate1e-05(44/90)
Experiment auto : baseline - AutoTypenameCudaW1024G50000MutationRate1e-06(45/90)
Experiment auto : baseline - AutoTypenameCpuW1024G500MutationRate0.0001(58/90)
Experiment auto : baseline - AutoTypenameCpuW1024G500MutationRate1e-05(59/90)
Experiment auto : baseline - AutoTypenameCpuW1024G500MutationRate1e-06(60/90)
Experiment auto : baseline - AutoTypena

KeyboardInterrupt: 